In [1]:
%cd ../airflow
%pwd

/home/welberthime/Development/fiap/cartolafc/airflow


'/home/welberthime/Development/fiap/cartolafc/airflow'

In [2]:
from include import RawExtractor

extractor = RawExtractor(
    base_url="https://api.github.com/repos/henriquepgomide/caRtola/contents/data",
    path="./data/raw",
)

extractor.extract_static_files("2019")
extractor.extract_dynamic_files("2019")

In [4]:
import pandas as pd
import uuid

In [6]:
partidas_df = pd.read_csv("./data/raw/2019/partidas/1.csv")

partidas_df.rename(
    columns={
        "home_team": "clubeMandanteID",
        "away_team": "clubeVisitanteID",
        "home_score": "mandantePlacar",
        "away_score": "visitantePlacar",
        "round": "rodada",
    },
    inplace=True,
)

def get_resultado(row):
    if row["mandantePlacar"] > row["visitantePlacar"]:
        return "Casa"
    elif row["mandantePlacar"] < row["visitantePlacar"]:
        return "Visitante"
    else:
        return "Empate"
    
partidas_df["resultado"] = partidas_df.apply(get_resultado, axis=1)
partidas_df["partidaID"] = [uuid.uuid4().hex for _ in range(len(partidas_df))]
partidas_df["temporada"] = 2019

partidas_df = partidas_df[[
    "partidaID",
    "clubeMandanteID",
    "clubeVisitanteID",
    "mandantePlacar",
    "visitantePlacar",
    "rodada",
    "resultado",
    "temporada",
]]

partidas_df

,partidaID,clubeMandanteID,clubeVisitanteID,mandantePlacar,visitantePlacar,rodada,resultado,temporada
0,4970a92afa5340edbe1ce3b6984fc7a1,282,314,2,1,1,Casa,2019
1,1625eb5da44347e0a1196175874b560a,315,285,2,0,1,Casa,2019
2,27539003cf48451789c08cd6f185c008,262,283,3,1,1,Casa,2019
3,2d4d2a112a4b4a459f283aca0c9aebe2,276,263,2,0,1,Casa,2019
4,f818ea1f915b448aa3689fbd51c03210,293,267,4,1,1,Casa,2019
...,...,...,...,...,...,...,...,...
375,338a1a872ba646158e2cb19535118f6d,290,284,3,2,38,Casa,2019
376,cc68605d2201474b94df266c95a3a9a6,285,282,2,1,38,Casa,2019
377,4038ca9c22d440aab27b74972df54f84,283,275,0,2,38,Visitante,2019
378,05e93bbe2f6547f989b8b0e11b4cbfd8,277,262,4,0,38,Casa,2019


In [10]:
from glob import glob

file_list = glob("./data/raw/2019/rodada/*.csv")
scouts_df = pd.DataFrame()
for file in file_list:
    tmp_df = pd.read_csv(file)
    scouts_df = pd.concat([scouts_df, tmp_df], sort=True)

scouts_df = scouts_df.rename(
    columns={
        "atletas.clube_id": "clubeID",
        "atletas.atleta_id": "atletaID",
        "atletas.rodada_id": "rodada",
        "atletas.pontos_num": "pontos",
        "atletas.media_num": "pontosMedia",
        "atletas.preco_num": "preco", 
        "atletas.variacao_num": "precoVariacao",
    },
)

house_by_rounds = partidas_df[["partidaID", "rodada", "clubeMandanteID"]].rename(
    columns={"clubeMandanteID": "clubeID"}
)
visitor_by_rounds = partidas_df[["partidaID", "rodada", "clubeVisitanteID"]].rename(
    columns={"clubeVisitanteID": "clubeID"}
)
club_by_rounds = pd.concat([house_by_rounds, visitor_by_rounds])

scouts_df = scouts_df.merge(
    right=club_by_rounds,
    on=["rodada", "clubeID"],
)

scouts_df = scouts_df[[
    "clubeID",
    "partidaID",
    "atletaID",
    "pontos",
    "pontosMedia",
    "preco",
    "precoVariacao",
    "FS",
    "PE",
    "A",
    "FT",
    "FD",
    "FF",
    "G",
    "I",
    "PP",
    "RB",
    "FC",
    "GC",
    "CA",
    "CV",
    "SG",
    "DD",
    "DP",
    "GS",
]]

scouts_df

,clubeID,partidaID,atletaID,pontos,pontosMedia,preco,precoVariacao,FS,PE,A,...,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,267,3ce3f3bf24d146dabd47aa50963fcb06,102139,1.90,1.77,2.89,0.13,4.0,2.0,NaN,...,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,267,3ce3f3bf24d146dabd47aa50963fcb06,102719,0.00,0.00,2.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,267,3ce3f3bf24d146dabd47aa50963fcb06,102852,0.00,0.00,3.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,267,3ce3f3bf24d146dabd47aa50963fcb06,102771,0.00,0.00,2.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,267,3ce3f3bf24d146dabd47aa50963fcb06,104678,-1.80,2.17,3.91,-1.59,NaN,2.0,NaN,...,NaN,1.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30576,265,d6f9d6ed2c474aa9ab1b58c1e83cbf88,79437,2.15,4.15,11.49,0.10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30577,265,d6f9d6ed2c474aa9ab1b58c1e83cbf88,68808,1.20,5.57,12.95,0.40,31.0,55.0,4.0,...,NaN,32.0,27.0,NaN,4.0,NaN,10.0,NaN,NaN,NaN
30578,265,d6f9d6ed2c474aa9ab1b58c1e83cbf88,78584,4.50,6.48,14.63,-0.52,7.0,7.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,14.0,54.0,1.0,27.0
30579,265,d6f9d6ed2c474aa9ab1b58c1e83cbf88,80189,-0.60,2.86,6.33,-0.35,10.0,47.0,1.0,...,NaN,15.0,7.0,NaN,2.0,NaN,14.0,NaN,NaN,NaN


In [11]:
posicoes_df = pd.read_csv("./data/raw/2019/posicoes_ids/1.csv")
posicoes_df.rename(
    columns={
        "Cod": "posicaoID",
        "Position": "nome",
        "abbr": "abreviacao",
    },
    inplace=True,
)

posicoes_df = posicoes_df[[
    "posicaoID",
    "nome",
    "abreviacao",
]]

posicoes_df

,posicaoID,nome,abreviacao
0,1,Goleiro,gol
1,2,Lateral,lat
2,3,Zagueiro,zag
3,4,Meia,mei
4,5,Atacante,ata
5,6,Técnico,tec


In [18]:
from glob import glob

file_list = glob("./data/raw/2019/rodada/*.csv")
atletas_df = pd.DataFrame()
for file in file_list:
    tmp_df = pd.read_csv(file)
    atletas_df = pd.concat([atletas_df, tmp_df], sort=True)

atletas_df.rename(
    columns={
        "atletas.clube_id": "clubeID",
        "atletas.atleta_id": "atletaID",
        "atletas.apelido": "apelido",
    },
    inplace=True,
)

atletas_df = atletas_df.merge(
    right=posicoes_df,
    left_on="atletas.posicao_id",
    right_on="abreviacao",
)

atletas_df = atletas_df[[
    "atletaID",
    "clubeID",
    "posicaoID",
    "apelido",
]]

atletas_df.drop_duplicates(inplace=True)
atletas_df.reset_index(drop=True, inplace=True)

atletas_df

,atletaID,clubeID,posicaoID,apelido
0,102139,267,5,Jairinho
1,101960,293,5,Vitinho
2,101997,293,5,Braian Romero
3,101885,276,5,Toró
4,102302,282,5,Papagaio
...,...,...,...,...
1070,106050,283,6,Ricardo Resende
1071,41258,283,6,Adilson Batista
1072,73317,354,6,Argel Fucks
1073,106683,341,6,Jacozinho


In [19]:
clubes_df = pd.read_csv("./data/raw/2019/times_ids/1.csv")
clubes_df.rename(
    columns={
        "id": "clubeID",
        "nome.cbf": "nome",
    },
    inplace=True,
)

clubes_df = clubes_df[["clubeID", "nome", "abreviacao"]]
clubes_df.drop_duplicates(subset=["clubeID"], keep="last", inplace=True)

clubes_df

,clubeID,nome,abreviacao
0,327,América - MG,AME
1,200,America - RN,OUT
2,373,Atlético - GO,ATL
3,282,Atlético - MG,ATL
6,293,Athlético-PR,ATL
7,314,Avaí - SC,AVA
8,265,Bahia - BA,BAH
9,263,Botafogo - RJ,BOT
10,203,Brasiliense,BRA
13,354,Ceará - CE,CEA
